You can select 'random' or 'select'. iF you select random, it will ask questions from all categories. If you select 'select' then you can select a keyword eg. 'crosstab', then it will ask questions only wrt the crosstab

In [4]:
import pandas as pd
import time  # For timing purposes
import numpy as np

# Load the Excel file
file_path = "data.xlsx"  # Change this to the path of your Excel file
questions_df = pd.read_excel(file_path)
questions_df.sample(frac=1)

def play_game(game_type="random"):
    keyword = None
    if game_type == "select":
        keyword = input("Please enter a keyword for the questions (e.g. 'crosstab'): ")

    # Rest of the code remains the same as in the simulation

    # List to keep track of asked questions and asked clusters
    asked_questions = []
    asked_clusters = []

    # Initialize the score
    score = 0

    def get_random_question():
        if game_type == "select" and keyword:
            # Filter based on keyword in the "Answer" column, ignoring NaN values
            available_questions = questions_df[
                (questions_df["Answer"].astype(str).str.contains(keyword, case=False, na=False)) & 
                (~questions_df["Questions"].isin(asked_questions))
            ]
            if available_questions.empty:
                return None
            # Randomly select a question from the filtered set
            question = available_questions.sample().iloc[0]
            asked_questions.append(question["Questions"])
            return question

        # For random game
        # Try to get a question from a cluster that hasn't been asked in the current rotation
        available_clusters = set(questions_df["Cluster"]) - set(asked_clusters)
        if not available_clusters:
            # Reset the clusters if all have been asked
            asked_clusters.clear()
            available_clusters = set(questions_df["Cluster"])

        chosen_cluster = np.random.choice(list(available_clusters))
        available_questions = questions_df[
            (~questions_df["Questions"].isin(asked_questions)) & 
            (questions_df["Cluster"] == chosen_cluster)
        ]

        if available_questions.empty:
            return None
        question = available_questions.sample().iloc[0]
        asked_questions.append(question["Questions"])
        return question

    # Ask 10 questions
    quit_flag = False  # Add this flag

    for i in range(10):
        if quit_flag:  # Check the flag at the start of each iteration
            break

        question = get_random_question()

        if question is None:
            print("No more available questions:")
            break

        asked_clusters.append(question["Cluster"])  # Keep track of asked clusters

        a = 0
        while True:
            print(f"\nQuestion {i + 1}: {question['Questions']}")

            user_answer = input("Your answer: ")

            correct_answer = question["Answer"]

            # Check if user wants to quit
            if user_answer.strip().lower() == "quit":
                print(f"\nYou chose to quit! Your current score is: {score} out of {i+1} questions asked.")
                quit_flag = True  # Set the flag to True
                break  # This will break out of the inner while loop

            if user_answer.strip() == correct_answer.strip():
                print(f"\nCorrect! (from {question['Cluster']} category)")
                print("==================================================")
                if a == 0:
                    score += 1
                    a = 0
                break
            else:
                print(f"\nSorry, the correct answer is: {correct_answer} (from {question['Cluster']} category)")
                a = 1
                break  # Exit the inner loop after providing the correct answer

    print(f"\nGame Over! Your final score is: {score} out of 10.")
    
    return score

# Initial prompt for game type
game_type = input("Do you want to play a random game or a select game? (Enter 'random' or 'select'): ").strip().lower()

play_game(game_type)


Do you want to play a random game or a select game? (Enter 'random' or 'select'): select
Please enter a keyword for the questions (e.g. 'crosstab'): loc

Question 1: How to get the last row
Your answer: df.loc[-1]

Sorry, the correct answer is: df.iloc[-1] (from DataFrame Creation and Inspection category)

Question 2: Get all the rows of 3rd, 5th and Last column use iloc
Your answer: quit

You chose to quit! Your current score is: 0 out of 2 questions asked.

Game Over! Your final score is: 0 out of 10.


0